In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Math

In [10]:
# Hệ số M đủ lớn ta sẽ dùng trong phương pháp Big-M ở bài này
M = 1000

# Đọc và chuẩn bị dữ liệu đầu vào
# Nếu muốn nhập trực tiếp test case vào đây để test thì dùng hàm này
def parse_input():
    """
    Hàm này định nghĩa bài toán tối ưu tuyến tính.
    
    Returns:
        tuple: (c, A, b, signs) - Hệ số hàm mục tiêu, ma trận hệ số, vế phải, dấu ràng buộc
    """
    # Bài toán tìm Max hàm f = 7x1 + x2 - 4x3 
    # Các ràng buộc:
    # -6x1 + 4x2 + 5x3 >= -20
    # x1 + 2x2 - x3 = 8
    # 3x1 + 2x2 - x3 <= -8
    c = [7, 1, -4]
    A = [
        [-6, 4, 5],
        [1, 2, -1],
        [3, 2, -1]
    ]
    b = [-20, 8, -8]
    signs = ['>=', '=', '<=']
    return c, A, b, signs




Nếu muốn đọc test case từ thư mục input thì ta sẽ dùng cell bên dưới đây. Sau đây sẽ là cấu trúc của test case ta sẽ sử dụng:

**Cấu trúc file testcase**
1. **Dòng 1**: Hệ số hàm mục tiêu `c`  
   - Ví dụ: `7 1 -4` (tương ứng với bài toán `Max f = 7x1 + x2 - 4x3`).

2. **Dòng 2 đến n+1**: Ma trận hệ số `A`  
   - Mỗi dòng là hệ số của một ràng buộc.  
   - Ví dụ:  
     ```
     -6 4 5
     1 2 -1
     3 2 -1
     ```

3. **Dòng tiếp theo**: Ký hiệu phân tách `b`  
   - Chỉ chứa ký tự `b`.
   - Dòng này được sử dụng để phân tách giữa ma trận hệ số `A` và vế phải `b` của các ràng buộc.

4. **Dòng tiếp theo**: Vế phải `b`  
   - Chứa các giá trị của vế phải trong các ràng buộc, cách nhau bằng dấu cách.
   - Ví dụ: `-20 8 -8`.

5. **Dòng cuối cùng**: Dấu ràng buộc  
   - Ví dụ: `>= = <=`.

---

**Ví dụ đầy đủ**
```plaintext
7 1 -4
-6 4 5
1 2 -1
3 2 -1
b
-20 8 -8
>= = <=
```

In [11]:
import os

# Nếu muốn đọc test case từ thư mục input thì dùng cell này
def parse_input_from_file(file_path):
    """
    Đọc dữ liệu bài toán từ file.
    
    Args:
        file_path (str): Đường dẫn tới file input.
    
    Returns:
        tuple: (c, A, b, signs) - Hệ số hàm mục tiêu, ma trận hệ số, vế phải, dấu ràng buộc.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Đọc hệ số hàm mục tiêu
    c = list(map(float, lines[0].strip().split()))
    
    # Đọc ma trận hệ số A
    A = []
    i = 1
    while lines[i].strip() != "b":
        A.append(list(map(float, lines[i].strip().split())))
        i += 1
    
    # Đọc vế phải b
    i += 1
    b = list(map(float, lines[i].strip().split()))
    
    # Đọc dấu ràng buộc
    i += 1
    signs = lines[i].strip().split()
    
    return c, A, b, signs

def parse_input_from_folder(folder_path):
    """
    Đọc dữ liệu từ tất cả các file trong thư mục.
    
    Args:
        folder_path (str): Đường dẫn tới thư mục chứa các file input.
    
    Returns:
        list: Danh sách các bài toán (c, A, b, signs).
    """
    problems = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            problems.append(parse_input_from_file(file_path))
    return problems

# Đọc dữ liệu từ thư mục input
input_folder = ".\input"
if not os.path.exists(input_folder):
    os.makedirs(input_folder)
problems = parse_input_from_folder(input_folder)

In [12]:
def format_number(num):
        return int(num) if num == int(num) else round(num, 2)

# Hiển thị bài toán
def display_problem(c, A, b, signs):
    """
    Hiển thị bài toán tối ưu tuyến tính dưới dạng dễ đọc
    """

    n = len(c)
    m = len(b)
    
    # Hiển thị hàm mục tiêu
    obj_func = "Max f = "
    for i in range(n):
        coef = format_number(c[i])
        if i > 0:
            if coef >= 0:
                obj_func += f" + {coef}x{i+1}" if coef != 1 else f" + x{i+1}"
            else:
                obj_func += f" - {abs(coef)}x{i+1}" if abs(coef) != 1 else f" - x{i+1}"
        else:
            obj_func += f"{coef}x{i+1}" if coef != 1 else f"x{i+1}"
    
    # Hiển thị các ràng buộc
    constraints = ["Subject to:"]
    for i in range(m):
        constraint = ""
        for j in range(n):
            coef = format_number(A[i][j])
            if j > 0:
                if coef >= 0:
                    constraint += f" + {coef}x{j+1}" if coef != 1 else f" + x{j+1}"
                else:
                    constraint += f" - {abs(coef)}x{j+1}" if abs(coef) != 1 else f" - x{j+1}"
            else:
                if coef >= 0:
                    constraint += f"{coef}x{j+1}" if coef != 1 else f"x{j+1}"
                else:
                    constraint += f"-{abs(coef)}x{j+1}" if abs(coef) != 1 else f"-x{j+1}"
        
        rhs = format_number(b[i])
        constraint += f" {signs[i]} {rhs}"
        constraints.append(constraint)
    
    # Hiển thị ràng buộc không âm
    non_neg = "x_j ≥ 0, j = 1,2,...," + str(n)
    
    # In ra kết quả
    display(Markdown(f"**Bài toán tối ưu tuyến tính:**"))
    display(Markdown(obj_func))
    for constraint in constraints:
        display(Markdown(constraint))
    display(Markdown(non_neg))

# Hiển thị từng bài toán từ thư mục input
for idx, (c, A, b, signs) in enumerate(problems):
    print(f"=== Bài toán {idx + 1} ===")
    display_problem(c, A, b, signs)

# Nếu dùng hàm parse_input() thì dùng đoạn này thay cho đoạn vừa rồi: 
# c, A, b, signs = parse_input()
# display_problem(c, A, b, signs)

=== Bài toán 1 ===


**Bài toán tối ưu tuyến tính:**

Max f = 7x1 + x2 - 4x3

Subject to:

-6x1 + 4x2 + 5x3 >= -20

...

Max f = 2x1 - x2 - 2x3

Subject to:

x1 + x2 - x3 = -1

x1 + 2x2 + x3 = 2

x_j ≥ 0, j = 1,2,...,3

In [13]:
# Tiền xử lý các ràng buộc

def preprocess_constraints(A, b, signs):
    """
    Tiền xử lý các ràng buộc để đảm bảo vế phải không âm.
    
    Args:
        A (list): Ma trận hệ số của các ràng buộc
        b (list): Vecto vế phải
        signs (list): Danh sách các dấu ràng buộc ('<=', '=', '>=')
        
    Returns:
        tuple: (new_A, new_b, new_signs) - Ma trận hệ số, vecto vế phải và dấu ràng buộc sau khi xử lý
    """
    new_A = []
    new_b = []
    new_signs = []

    for i in range(len(A)):
        row = A[i].copy()  # Tạo bản sao để tránh thay đổi dữ liệu gốc
        rhs = b[i]
        sign = signs[i]

        # Đưa RHS về không âm
        if rhs < 0:
            row = [-x for x in row]
            rhs = -rhs
            if sign == '<=':
                sign = '>='
            elif sign == '>=':
                sign = '<='

        new_A.append(row)
        new_b.append(rhs)
        new_signs.append(sign)

    return new_A, new_b, new_signs

# Test preprocessing
new_A, new_b, new_signs = preprocess_constraints(A, b, signs)

# Hiển thị kết quả preprocessing
print("=== Sau khi tiền xử lý ===")
for i in range(len(new_A)):
    row_str = " + ".join([f"{coef}x{j+1}" if j == 0 or coef >= 0 else f"- {abs(coef)}x{j+1}" 
                           for j, coef in enumerate(new_A[i]) if coef != 0])
    print(f"{row_str} {new_signs[i]} {new_b[i]}")

=== Sau khi tiền xử lý ===
-1.0x1 + - 1.0x2 + 1.0x3 = 1.0
1.0x1 + 2.0x2 + 1.0x3 = 2.0


In [14]:
# Xây dựng bảng đơn hình Big-M

def build_bigM_tableau(c, A, b, signs):
    """
    Xây dựng bảng đơn hình ban đầu cho phương pháp Big-M.
    
    Args:
        c (list): Hệ số hàm mục tiêu
        A (list): Ma trận hệ số của các ràng buộc
        b (list): Vecto vế phải
        signs (list): Danh sách các dấu ràng buộc ('<=', '=', '>=')
        
    Returns:
        tuple: (basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)
    """
    A, b, signs = preprocess_constraints(A, b, signs)

    num_orig_vars = len(c)
    tableau = []
    basis = []
    basis_cost = []
    var_names = [f"x{i+1}" for i in range(num_orig_vars)]
    c_extended = c[:]  # Bắt đầu với hệ số gốc

    for i, (row, sign) in enumerate(zip(A, signs)):
        row_extended = row[:]

        if sign == '<=':
            # Thêm biến slack
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(0)
            basis.append(len(var_names) - 1)
            basis_cost.append(0)

        elif sign == '>=':
            # Thêm biến surplus và artificial
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(-1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(0)

            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(-M)
            basis.append(len(var_names) - 1)
            basis_cost.append(-M)

        elif sign == '=':
            # Thêm artificial
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(-M)
            basis.append(len(var_names) - 1)
            basis_cost.append(-M)

        tableau.append(row_extended)

    # Đệm 0 để làm đều các dòng
    total_vars = len(var_names)
    for row in tableau:
        row += [0] * (total_vars - len(row))

    # Cách tính z_row dựa trên định nghĩa
    z_row = [0] * total_vars
    rhs_z = 0
    
    # Tính RHS của hàng Z: Σ(cBi·bi)
    for i in range(len(tableau)):
        rhs_z += basis_cost[i] * b[i]
    
    # Với mỗi biến j, tính Δj = Σ(cBi·aij) - cj
    for j in range(total_vars):
        # Tính Σ(cBi·aij)
        sum_cb_aij = 0
        for i in range(len(tableau)):
            sum_cb_aij += basis_cost[i] * tableau[i][j]
        
        # Trừ đi cj
        z_row[j] = sum_cb_aij - c_extended[j]

    return basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended

# Xây dựng bảng từ dữ liệu đầu vào
basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended = build_bigM_tableau(c, A, b, signs)

In [15]:
# Hiển thị bảng đơn hình Big-M
def print_bigM_tableau(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended):
    """
    Hiển thị bảng đơn hình cho phương pháp Big-M.
    
    Args:
        basis (list): Chỉ số của các biến trong cơ sở
        basis_cost (list): Chi phí của các biến trong cơ sở
        b (list): Vecto vế phải
        tableau (list): Bảng đơn hình
        z_row (list): Hàng Z-row (reduced costs)
        rhs_z (float): Giá trị hàm mục tiêu
        var_names (list): Tên của các biến
        c_extended (list): Hệ số chi phí mở rộng
    """

    print("\n=== Bảng đơn hình ban đầu (phương pháp Big-M) ===\n")

    num_vars = len(var_names)
    header = ["Biến cơ sở", "Hệ số", "Phương án"] + var_names
    print(" | ".join(f"{h:^12}" for h in header))
    print("-" * (14 * len(header)))

    # Hiển thị hệ số hàm mục tiêu (bao gồm cả artificial variables)
    c_row = [" " * 12, " " * 12, " " * 12] + [f"{format_number(coef):^12}" for coef in c_extended]
    print(" | ".join(c_row))
    print("-" * (14 * len(header)))

    for i in range(len(basis)):
        var = var_names[basis[i]]
        coef = format_number(basis_cost[i])
        row_vals = [format_number(v) for v in tableau[i]]
        row_str = [f"{var:^12}", f"{coef:^12}", f"{format_number(b[i]):^12}"] + [f"{v:^12}" for v in row_vals]
        print(" | ".join(row_str))

    # Hiển thị toàn bộ z_row
    z_row_vals = [format_number(v) for v in z_row]
    z_row_str = ["Z".center(12), " ".center(12), f"{format_number(rhs_z):^12}"] + [f"{v:^12}" for v in z_row_vals]
    print("-" * (14 * len(header)))
    print(" | ".join(z_row_str))

# Hiển thị bảng đơn hình dạng Pandas DataFrame
def display_tableau_df(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended):
    """
    Hiển thị bảng đơn hình dưới dạng DataFrame cho dễ đọc.
    """
    # Tạo DataFrame cho bảng chính
    data = []
    for i in range(len(basis)):
        row_data = {
            'Biến cơ sở': var_names[basis[i]],
            'Hệ số': format_number(basis_cost[i]),
            'Phương án': format_number(b[i])
        }
        for j, name in enumerate(var_names):
            row_data[name] = format_number(tableau[i][j])
        data.append(row_data)
    
    # Thêm hàng Z
    z_data = {
        'Biến cơ sở': 'Z',
        'Hệ số': '',
        'Phương án': format_number(rhs_z)
    }
    for j, name in enumerate(var_names):
        z_data[name] = format_number(z_row[j])
    data.append(z_data)
    
    # Tạo DataFrame
    df = pd.DataFrame(data)
    
    # Hiển thị các hệ số hàm mục tiêu
    display(Markdown("**Hệ số hàm mục tiêu (c):**"))
    c_data = {}
    for j, name in enumerate(var_names):
        c_data[name] = format_number(c_extended[j])
    c_df = pd.DataFrame([c_data])
    display(c_df)
    
    display(Markdown("**Bảng đơn hình:**"))
    return df

# Hiển thị bảng đơn hình
print_bigM_tableau(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)

# Hiển thị bảng đơn hình dạng DataFrame
tableau_df = display_tableau_df(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)
display(tableau_df)


=== Bảng đơn hình ban đầu (phương pháp Big-M) ===

 Biến cơ sở  |    Hệ số     |  Phương án   |      x1      |      x2      |      x3      |      x4      |      x5     
----------------------------------------------------------------------------------------------------------------
             |              |              |      2       |      -1      |      -2      |    -1000     |    -1000    
----------------------------------------------------------------------------------------------------------------
     x4      |    -1000     |      1       |      -1      |      -1      |      1       |      1       |      0      
     x5      |    -1000     |      2       |      1       |      2       |      1       |      0       |      1      
----------------------------------------------------------------------------------------------------------------
     Z       |              |    -3000     |      -2      |     -999     |    -1998     |      0       |      0      


**Hệ số hàm mục tiêu (c):**

,x1,x2,x3,x4,x5
0,2,-1,-2,-1000,-1000


**Bảng đơn hình:**

,Biến cơ sở,Hệ số,Phương án,x1,x2,x3,x4,x5
0,x4,-1000,1,-1,-1,1,1,0
1,x5,-1000,2,1,2,1,0,1
2,Z,,-3000,-2,-999,-1998,0,0


Nếu muốn từ một folder input gồm nhiều file testcase khác nhau dạng .txt, ta sẽ dùng hàm ở cell dưới đây để xuát ra kết quả các file output dạng .txt với những bảng đơn hình xuất phát tương ứng.

In [16]:
import os

def save_tableau_to_file(output_folder, file_name, basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended):
    """
    Lưu bảng đơn hình vào file TXT trong thư mục output.
    """
    # Tạo thư mục output nếu chưa tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Tạo nội dung bảng đơn hình dưới dạng chuỗi
    output_lines = []
    header = ["Biến cơ sở", "Hệ số", "Phương án"] + var_names
    output_lines.append(" | ".join(f"{h:^12}" for h in header))
    output_lines.append("-" * (14 * len(header)))

    # Hệ số hàm mục tiêu
    c_row = [" " * 12, " " * 12, " " * 12] + [f"{format_number(coef):^12}" for coef in c_extended]
    output_lines.append(" | ".join(c_row))
    output_lines.append("-" * (14 * len(header)))

    # Các dòng của bảng đơn hình
    for i in range(len(basis)):
        var = var_names[basis[i]]
        coef = format_number(basis_cost[i])
        row_vals = [format_number(v) for v in tableau[i]]
        row_str = [f"{var:^12}", f"{coef:^12}", f"{format_number(b[i]):^12}"] + [f"{v:^12}" for v in row_vals]
        output_lines.append(" | ".join(row_str))

    # Hàng Z-row
    z_row_vals = [format_number(v) for v in z_row]
    z_row_str = ["Z".center(12), " ".center(12), f"{format_number(rhs_z):^12}"] + [f"{v:^12}" for v in z_row_vals]
    output_lines.append("-" * (14 * len(header)))
    output_lines.append(" | ".join(z_row_str))

    # Lưu nội dung vào file TXT
    output_file = os.path.join(output_folder, f"{file_name}_output.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(output_lines))
    print(f"Bảng đơn hình đã được lưu vào: {output_file}")

# Xử lý từng file testcase trong thư mục input
input_folder = "./input"
output_folder = "./output"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for file_name in os.listdir(input_folder):
    if file_name.endswith('.txt'):
        file_path = os.path.join(input_folder, file_name)
        print(f"Đang xử lý file: {file_name}")

        # Đọc dữ liệu từ file
        c, A, b, signs = parse_input_from_file(file_path)

        # Xây dựng bảng đơn hình Big-M
        basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended = build_bigM_tableau(c, A, b, signs)

        # Hiển thị bảng đơn hình
        print_bigM_tableau(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)

        # Lưu bảng đơn hình vào file output
        save_tableau_to_file(output_folder, file_name.split('.')[0], basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)

Đang xử lý file: test1.txt

=== Bảng đơn hình ban đầu (phương pháp Big-M) ===

 Biến cơ sở  |    Hệ số     |  Phương án   |      x1      |      x2      |      x3      |      x4      |      x5      |      x6      |      x7     
--------------------------------------------------------------------------------------------------------------------------------------------
             |              |              |      7       |      1       |      -4      |      0       |    -1000     |      0       |    -1000    
--------------------------------------------------------------------------------------------------------------------------------------------
     x4      |      0       |      20      |      6       |      -4      |      -5      |      1       |      0       |      0       |      0      
     x5      |    -1000     |      8       |      1       |      2       |      -1      |      0       |      1       |      0       |      0      
     x7      |    -1000     |      8       |   